In [1]:
# IMPORTING LIBRARIES
import pandas as pd
import numpy as np
import re

import nltk
import sys

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from fuzzywuzzy import fuzz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer,classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score, KFold,cross_validate
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)


C:\Users\P.Vijay Srinivasan\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# loading data
df = pd.read_csv("hcp_train.csv", encoding="latin-1")
df
test=pd.read_csv("hcp_test.csv",encoding="latin-1")
df

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0
4,1005,Mobile,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,Houston,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal,Website,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113932,114933,Desktop,2,68.82.97.126,051e4a06-4cb4-4b97-b0b1-d1c6a2b8e2c2,Philadelphia,19120.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/consultat...,Cardiovascular|Clinical|False|Male|Oncology|Ps...,207RP1001X,1.0
113933,114934,Desktop,2,104.172.11.109,3289bba5-4a50-4b61-a5cc-68596bbdc2bf,Van Nuys,914014213.0,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/features/...,Asthma|Arthritis|General|Clinical|Diagnosis|Rh...,2084P0800X,1.0
113934,114935,Desktop,7,174.21.94.113,9c36e90c-caa7-4323-8431-fb84f8293972,NaN,98105.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/39455-whats-th...,Rehabilitation|Transplantation|Clinical|Preven...,208000000X,1.0
113935,114936,Mobile,2,69.253.129.131,afc27760-31f7-4f6e-aa08-90d8a96525d0,Wilmington,19808.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,Website,https://www.dermatologyadvisor.com/home/topics...,pain|disease|autoimmune|multiple sclerosis|gra...,2084P0800X,1.0


In [3]:
df.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY            6359
USERZIPCODE         4592
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81624
IS_HCP                 1
dtype: int64

In [4]:
# identified null calues and using mode to fill categorical colmns
null_columns = ["USERCITY", "USERZIPCODE", "USERAGENT"]

for i in null_columns:
    mode = df[i].mode().iloc[0]  # Extract the mode value from the Series
    df[i].fillna(mode, inplace=True)
    print(mode)

New York
63169.0
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36


In [5]:
# jsut checking for duplicates if any
df=df.drop_duplicates()
# checking null values for platformuid column to check if these rows are necessary or not 
df[df["USERPLATFORMUID"].isnull()]
# its seen that they are hcp's and crucial for model output and taxonomy prediction so we decided to keep it intact.

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
59158,60159,Desktop,2,152.11.230.114,NaN,Winston-Salem,27103.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,Urology|Chronic|Gastroenterology|Ophthalmology...,2084P0800X,1.0
80923,81924,Desktop,7,73.59.230.146,NaN,Shreveport,71106.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/39455-whats-th...,Rehabilitation|Transplantation|Clinical|Preven...,207RG0300X,1.0
99416,100417,Desktop,2,152.11.230.114,NaN,Winston-Salem,27103.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,Urology|Chronic|Gastroenterology|Ophthalmology...,2084P0800X,1.0
102512,103513,Desktop,7,73.59.230.146,NaN,Shreveport,71106.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/39455-whats-th...,Rehabilitation|Transplantation|Clinical|Preven...,207RG0300X,1.0


In [6]:
df.dropna(subset=["IS_HCP"], inplace=True)

In [7]:
# printing unique values
for i in df.columns:
    print(i,":",df[i].unique())
    print(" ")
    print(len(df[i].unique()))

ID : [  1001   1002   1003 ... 114935 114936 114937]
 
113936
DEVICETYPE : ['Desktop' 'Mobile' 'Tablet' 'Unknown']
 
4
PLATFORM_ID : [ 2  3  7  5  9  6  4 10 14 12  8 15  1 13 16]
 
15
BIDREQUESTIP : ['170.173.0.22' '65.216.253.25' '66.232.79.22' ... '68.82.97.126'
 '174.21.94.113' '108.41.233.175']
 
33664
USERPLATFORMUID : ['6974dcaa-f932-480e-9fb5-c52e20e1393a'
 'c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf'
 'a698de4b-e200-46dd-b5fb-40402175ae18' ...
 '9c36e90c-caa7-4323-8431-fb84f8293972'
 'afc27760-31f7-4f6e-aa08-90d8a96525d0'
 'd26887c5-15f9-4b72-80dc-728039529e60']
 
53781
USERCITY : ['Portland' 'Arlington' 'New Meadows' ... 'Texhoma' 'Goodhue' 'Gambier']
 
4420
USERZIPCODE : [9.72060000e+04 2.22020000e+04 8.36540000e+04 ... 4.34500000e+04
 7.87356202e+08 7.60865488e+08]
 
11278
USERAGENT : ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like

In [8]:
df=df.drop("CHANNELTYPE",axis=1)
# only one value this column does not provide any useful information for the analysis. 
# Having a single value throughout the entire column does not contribute to the variability of the data and 
# does not provide any discriminatory power.

In [9]:
# doing same opearation for test data
test=test.drop("CHANNELTYPE",axis=1)
test

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,URL,KEYWORDS
0,115501,Desktop,2,75.189.231.103,0d5041ff-f0b6-4d1a-9ad7-0a29f7d485b4,Fayetteville,28305.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,Online Medical Journal,https://www.clinicaladvisor.com/home/features/...,Family Practice|Drainage|Clinical|Dermatology|...
1,115502,Mobile,2,24.101.33.158,c8396dd0-969f-4d99-a40b-b7bb1f516154,Conneaut Lake,16316.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,https://www.ophthalmologyadvisor.com/topics/ca...,General|Clinical|Operative|Medicine|Cardiology...
2,115503,Desktop,2,172.118.216.142,3c97a081-6518-43f8-9f26-369759cfb471,Covina,91724.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,https://www.psychiatryadvisor.com/author/tori-...,Abortion|Anxiety Disorders|Apnea|False|Trauma|...
3,115504,Desktop,7,71.105.120.171,3e2578c8-f794-41af-a38c-c5cfb3c0f014,Brooklyn,11226.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,https://www.cureus.com/articles/105482-diverti...,Health|Male|Neurological Surgery|Otolaryngolog...
4,115505,Desktop,2,73.82.211.73,ec2ae7ce-6a8c-4156-98a7-07203e60f483,Marietta,30062.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,https://www.renalandurologynews.com/home/confe...,chronic kidney disease|pain|nephrology|disease...
...,...,...,...,...,...,...,...,...,...,...,...
28488,143989,Desktop,2,69.202.233.241,78ce4bbe-3885-4c14-b945-a0ea1e4574f4,Brooklyn,11215.0,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,Online Medical Journal,https://www.renalandurologynews.com/home/news/...,Transplantation|Psychiatry|Heart Failure|Angio...
28489,143990,Desktop,7,75.4.190.65,a53799e1-b279-40cf-b467-11b4d659e325,Miami,33178.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,https://www.cureus.com/publish/articles/144818...,small|hospitals|nephrology|emergency medicine|...
28490,143991,Desktop,7,137.52.180.45,acc732a8-2ee9-4e2c-9933-70e34db48101,Fort Lauderdale,33314.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,https://www.cureus.com/registrations/continue?...,small|hospitals|nephrology|emergency medicine|...
28491,143992,Desktop,8,66.249.66.4,dce21294-b105-4abb-b145-4e62d71def44,NaN,NaN,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,Online Medical Journal,https://radrounds.com/radiology-case-images-te...,the|disease|pain|radiology|lung abscess|dyspar...


In [10]:
# removing unnecessary columns in train and test dataset
cols_to_remove = ["USERCITY", "USERZIPCODE", "USERAGENT", "PLATFORM_ID", "BIDREQUESTIP","USERPLATFORMUID"]
df = df.drop(cols_to_remove, axis=1)

In [11]:
test=test.drop(cols_to_remove, axis=1)

In [12]:
test

,ID,DEVICETYPE,PLATFORMTYPE,URL,KEYWORDS
0,115501,Desktop,Online Medical Journal,https://www.clinicaladvisor.com/home/features/...,Family Practice|Drainage|Clinical|Dermatology|...
1,115502,Mobile,Online Medical Journal,https://www.ophthalmologyadvisor.com/topics/ca...,General|Clinical|Operative|Medicine|Cardiology...
2,115503,Desktop,Online Medical Journal,https://www.psychiatryadvisor.com/author/tori-...,Abortion|Anxiety Disorders|Apnea|False|Trauma|...
3,115504,Desktop,Online Medical Journal,https://www.cureus.com/articles/105482-diverti...,Health|Male|Neurological Surgery|Otolaryngolog...
4,115505,Desktop,Online Medical Journal,https://www.renalandurologynews.com/home/confe...,chronic kidney disease|pain|nephrology|disease...
...,...,...,...,...,...
28488,143989,Desktop,Online Medical Journal,https://www.renalandurologynews.com/home/news/...,Transplantation|Psychiatry|Heart Failure|Angio...
28489,143990,Desktop,Online Medical Journal,https://www.cureus.com/publish/articles/144818...,small|hospitals|nephrology|emergency medicine|...
28490,143991,Desktop,Online Medical Journal,https://www.cureus.com/registrations/continue?...,small|hospitals|nephrology|emergency medicine|...
28491,143992,Desktop,Online Medical Journal,https://radrounds.com/radiology-case-images-te...,the|disease|pain|radiology|lung abscess|dyspar...


In [13]:
df["URL"][0]

'https://www.cancertherapyadvisor.com/home/cancer-topics/prostate-cancer/'

In [14]:
df.iloc[3][4]

'Dental|Total|Clinical|Pharmacology|Physicians|Health|Drug Therapy|Oncology|False'

In [15]:
# URL keywords extraction for both train and test datasets
from urllib.parse import urlparse

common_words = ['http:', 'https:', 'www', 'com', 'home', 'html', 'htm', 'php', 'asp', 'aspx', 'jsp', 'index', 'default', 'blog', 'article', 'articles', 'category', "#","39455",'after','a','to','the']
custom_stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def extract_keywords(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc.replace('www.', '')
    path = parsed_url.path.strip('/')

    subdomains = parsed_url.hostname.split('.')
    subdomains = [subdomain for subdomain in subdomains if subdomain.lower() not in common_words]

    path_keywords = [word.lower() for word in re.split('[/-]', path) if word and word.lower() not in common_words]
    path_keywords = [token for token in path_keywords if re.match(r'^[a-zA-Z0-9]+$', token)]
    path_keywords = [lemmatizer.lemmatize(token) for token in path_keywords]

    domain_keywords = [word.lower() for word in re.split('[/-]', domain) if word and word.lower() not in common_words]
    domain_keywords = [token for token in domain_keywords if re.match(r'^[a-zA-Z0-9]+$', token)]
    domain_keywords = [lemmatizer.lemmatize(token) for token in domain_keywords]

    
    keywords = subdomains + domain_keywords + path_keywords
    
    keywords_strings = ', '.join(keywords)
    
    return keywords_strings


In [16]:
def remove_duplicates(keywords_string, custom_stopwords):
    tokens = keywords_string.split()
    unique_tokens = set()

    for token in tokens:
        lowercase_token = token.lower()
        if lowercase_token not in custom_stopwords:
            unique_tokens.add(lowercase_token)

    return ' '.join(unique_tokens)

In [17]:
# removing duplicate or repeating keywords in URL'S
df['url_terms'] = df['URL'].apply(lambda x: remove_duplicates(extract_keywords(x),custom_stopwords))
test["url_terms"] = test['URL'].apply(lambda x: remove_duplicates(extract_keywords(x),custom_stopwords))

In [18]:
df["KEYWORDS"][113932]

'Cardiovascular|Clinical|False|Male|Oncology|Psychiatry|Medicine|Hepatology|General|Total|Gastroenterology|Neurology|Urology|Health|Cardiology|Asthma|Orthopedics|Pediatrics|Rheumatology|Dermatology|Surgery|Acquired|Ophthalmology'

In [19]:
df["url_terms"][108]
test["url_terms"][199]

'duct, hamartoma, review biliary, cureus, 98184, systematic,'

In [20]:
df["url_terms"][113932]
df["url_terms"].head(1)

0    cancer prostate, cancertherapyadvisor, cancer,...
Name: url_terms, dtype: object

In [21]:
len(set(df["url_terms"]))

2651

In [22]:
#removing symbols from KEYWORDS column
def preprocess_keywords(row):
    keywords = row['KEYWORDS']
    # Remove special characters (except '|') using regular expressions
    keywords = re.sub(r'[^\w\s|]', '', keywords)
    return keywords

# Apply preprocessing to the 'KEYWORDS' column
df['PREPROCESSED_KEYWORDS'] = df.apply(preprocess_keywords, axis=1)
test['PREPROCESSED_KEYWORDS'] = df.apply(preprocess_keywords, axis=1)

In [23]:
df['PREPROCESSED_KEYWORDS'][113932]

'Cardiovascular|Clinical|False|Male|Oncology|Psychiatry|Medicine|Hepatology|General|Total|Gastroenterology|Neurology|Urology|Health|Cardiology|Asthma|Orthopedics|Pediatrics|Rheumatology|Dermatology|Surgery|Acquired|Ophthalmology'

In [24]:
# USING WORD2VEC MODEL ON KEYWORDS COLUMN FOR NUMBERICAL VECTOR REPRESENATION
from gensim.models import Word2Vec

sentences = [keywords.split('|') for keywords in df['KEYWORDS']]
test_sentences = [keywords.split('|') for keywords in test['KEYWORDS']]
# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1, epochs=10)
model1=Word2Vec(test_sentences, vector_size=100, window=5, min_count=1, sg=1, epochs=10)

In [25]:
# saving models
model.save("my_word2vec_model.bin")
model1.save("my_word2vec_model1.bin")

In [26]:
# CREATING VECTOR REPRESENTATION FOR ALL KEYWORDS
keyword_vector = {}
keyword_vector1 = {}

def get_keyword_vector(row, model):
    keywords = row['PREPROCESSED_KEYWORDS']
    tokens = keywords.split('|')
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        for i in tokens:
            keyword_vector[i] = sum(vectors) / len(vectors)  # Assign the vector to the dictionary
            return keyword_vector[i]
        else:
            return None
def get_keyword_vector1(row, model):
    keywords = row['PREPROCESSED_KEYWORDS']
    tokens = keywords.split('|')
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        for i in tokens:
            keyword_vector1[i] = sum(vectors) / len(vectors)  # Assign the vector to the dictionary
            return keyword_vector1[i]
        else:
            return None

# Load the Word2Vec model
model = Word2Vec.load("my_word2vec_model.bin")
model1=Word2Vec.load("my_word2vec_model1.bin")
# Example usage
df['KEYWORD_VECTORS'] = df.apply(lambda row: get_keyword_vector(row, model), axis=1)
test['KEYWORD_VECTORS'] =test.apply(lambda row: get_keyword_vector1(row, model1), axis=1)

In [27]:
# keyword vector for train data
keyword_vector

{'General': array([ 0.07425748,  0.2148097 , -0.14351544, -0.1586284 ,  0.16520478,
        -0.05241118,  0.21472281,  0.61058503, -0.15232867, -0.07702728,
         0.1878464 , -0.32652465, -0.00194048,  0.12030993,  0.2531358 ,
         0.08998472, -0.00287741, -0.21643497, -0.17373465, -0.5292196 ,
         0.23533991,  0.18187223,  0.13533615, -0.1505036 , -0.29768714,
         0.01720751, -0.3599374 , -0.07317765,  0.05987115,  0.11408166,
         0.18571395, -0.23589739, -0.13072073, -0.10345285, -0.3092688 ,
        -0.05333584, -0.00924568, -0.17556658,  0.22027914, -0.43930104,
         0.45905635, -0.03990025,  0.00793559,  0.16866645,  0.09772234,
        -0.10728482, -0.37835082, -0.08250306,  0.08714733,  0.12988125,
         0.07140333,  0.05834563,  0.04783516, -0.00852136, -0.07600785,
        -0.10944397,  0.49491173,  0.05769522, -0.13067205,  0.1497436 ,
         0.02546669,  0.02949462, -0.05040802, -0.04017427, -0.23269947,
         0.4434914 , -0.00581825,  0.221

In [28]:
# keyword vector for train data
keyword_vector1

{'General': array([-0.0994343 ,  0.23626044,  0.06313471,  0.27639925, -0.18696675,
        -0.3181883 ,  0.15480773,  0.73681253, -0.45350197, -0.18834822,
         0.0285441 , -0.18170506, -0.14215714,  0.15110625,  0.07180658,
         0.17822994,  0.39513856, -0.06081693, -0.03061447, -0.5768645 ,
        -0.05769561,  0.03800605, -0.03569662, -0.04284023, -0.33275023,
         0.21526022, -0.16476554,  0.05398061, -0.11049511,  0.15334065,
         0.3505815 , -0.03092033,  0.04594718, -0.14517862, -0.18507771,
         0.04810113,  0.17230065,  0.03759353,  0.01831684, -0.24803795,
         0.25081295, -0.1243788 , -0.05821806,  0.14398283,  0.11787495,
        -0.20521384, -0.37656847, -0.06776101,  0.16174199, -0.2437647 ,
         0.01016431, -0.04346409, -0.0588652 , -0.04852492,  0.14236718,
        -0.19355497,  0.2699836 , -0.22769907, -0.00936311,  0.17008941,
        -0.2572377 ,  0.2276286 ,  0.2340551 , -0.09703996, -0.16540764,
         0.5159995 ,  0.02229549,  0.244

In [29]:
df['KEYWORD_VECTORS']

0         [-0.019790778, 0.33817682, -0.19910386, -0.182...
1         [-0.06805519, 0.32602564, -0.18580012, -0.1181...
2         [-0.019790778, 0.33817682, -0.19910386, -0.182...
3         [-0.16594602, 0.3015865, -0.23950914, -0.23330...
4         [0.0872032, 0.2885589, -0.16864707, -0.2295782...
                                ...                        
113932    [0.008820659, 0.2892336, -0.16918428, -0.14777...
113933    [0.06199405, 0.28575888, -0.18397531, -0.11896...
113934    [-0.021794908, 0.3659684, -0.13851866, -0.2263...
113935    [0.056872275, 0.3604982, 0.1038318, 0.10823041...
113936    [0.008664939, 0.31802, -0.14026071, -0.0608784...
Name: KEYWORD_VECTORS, Length: 113936, dtype: object

In [30]:
df["PREPROCESSED_KEYWORDS"][113932]

'Cardiovascular|Clinical|False|Male|Oncology|Psychiatry|Medicine|Hepatology|General|Total|Gastroenterology|Neurology|Urology|Health|Cardiology|Asthma|Orthopedics|Pediatrics|Rheumatology|Dermatology|Surgery|Acquired|Ophthalmology'

In [31]:
df

,ID,DEVICETYPE,PLATFORMTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP,url_terms,PREPROCESSED_KEYWORDS,KEYWORD_VECTORS
0,1001,Desktop,Online Medical Journal,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0,"cancer prostate, cancertherapyadvisor, cancer,...",General|Endocrine|False|Medicine|Surgery|Urolo...,"[-0.019790778, 0.33817682, -0.19910386, -0.182..."
1,1002,Desktop,Online Medical Journal,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0,"medicine, zone, support, hematology, marginal,...",Bone Marrow|Radiography|Chronic|Oncology|Psych...,"[-0.06805519, 0.32602564, -0.18580012, -0.1181..."
2,1003,Desktop,Online Medical Journal,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0,"cancer prostate, cancertherapyadvisor, cancer,...",General|Endocrine|False|Medicine|Surgery|Urolo...,"[-0.019790778, 0.33817682, -0.19910386, -0.182..."
3,1004,Desktop,Online Medical Journal,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0,"warfarin, globalrph, consult, tool maintenance...",Dental|Total|Clinical|Pharmacology|Physicians|...,"[-0.16594602, 0.3015865, -0.23950914, -0.23330..."
4,1005,Mobile,Online Medical Journal,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,0.0,"cystic, transmembrane, of, fibrosis, 58184, tr...",Critical Care|Emergency Medicine|General Pract...,"[0.0872032, 0.2885589, -0.16864707, -0.2295782..."
...,...,...,...,...,...,...,...,...,...,...
113932,114933,Desktop,Online Medical Journal,https://www.clinicaladvisor.com/home/consultat...,Cardiovascular|Clinical|False|Male|Oncology|Ps...,207RP1001X,1.0,"relationship hpv, clinicaladvisor, consultatio...",Cardiovascular|Clinical|False|Male|Oncology|Ps...,"[0.008820659, 0.2892336, -0.16918428, -0.14777..."
113933,114934,Desktop,Online Medical Journal,https://www.clinicaladvisor.com/home/features/...,Asthma|Arthritis|General|Clinical|Diagnosis|Rh...,2084P0800X,1.0,"dx derm, clinicaladvisor, feature,",Asthma|Arthritis|General|Clinical|Diagnosis|Rh...,"[0.06199405, 0.28575888, -0.18397531, -0.11896..."
113934,114935,Desktop,Online Medical Journal,https://www.cureus.com/articles/39455-whats-th...,Rehabilitation|Transplantation|Clinical|Preven...,208000000X,1.0,"hazard, odds, ratio, ratio differentiating, an...",Rehabilitation|Transplantation|Clinical|Preven...,"[-0.021794908, 0.3659684, -0.13851866, -0.2263..."
113935,114936,Mobile,Online Medical Journal,https://www.dermatologyadvisor.com/home/topics...,pain|disease|autoimmune|multiple sclerosis|gra...,2084P0800X,1.0,"dermatitis, disorder atopic, for, autoimmune, ...",pain|disease|autoimmune|multiple sclerosis|gra...,"[0.056872275, 0.3604982, 0.1038318, 0.10823041..."


In [32]:
# creating a taxonomy map to fill taxonomies where same keywords matches
keyword_taxonomy_map = {}
taxonomy_map={}
url_map={}
for index, row in df[df["IS_HCP"]==1].iterrows():
    keywords = row['KEYWORDS']
    taxonomy = row['TAXONOMY']
    url = row['URL']
    if url not in url_map:
        url_map[url] = taxonomy
    keyword_list = keywords.split('|')

    for keyword in keywords.split('|'):
        if keyword not in keyword_taxonomy_map:
            keyword_taxonomy_map[keyword] = [taxonomy]  # Create a new list with the first taxonomy
        else:
            if taxonomy not in keyword_taxonomy_map[keyword]:
                keyword_taxonomy_map[keyword].append(taxonomy)

        if keywords not in taxonomy_map:
            taxonomy_map[keywords] = taxonomy
# print(keyword_taxonomy_map)
taxonomy_map

{'Dental|Total|Clinical|Pharmacology|Physicians|Health|Drug Therapy|Oncology|False': '2084P0800X',
 'Intensive Care|Shock|Ischemic Heart Disease|Health|Drainage|Liver Transplantation|Transplantation|Controlled|Urology|Critical Care|Erythema|Malnutrition|Hepatology|Hypotension|General|Intrahepatic|Small|Neurology|Lung|Cardiac|Oral|Chronic|Total|Tachycardia|Liver|Angiotensin|Kidney Transplantation|Diagnosis|Endocrine|False|Nervous System|Cardiology|Critical Care Medicine|Ophthalmology|Resuscitation|Adrenal Insufficiency|Intravenous|Metabolic|Gastroenterology|Cardiac Output|Hemorrhage|Diabetes Mellitus|Clinical|Induced|Oncology|Psychiatry|Medicine|Hematuria|Myocardial Ischemia|Respiratory Tract|Intrinsic|Nephritis|Rheumatology|Perfusion|Cardiomyopathy|Refractory|Large|Dermatology': '2084P0804X',
 'Gastrointestinal Tract|Female|Oncology|Medicine|Small|Metabolic|Elderly|Health|False|Trauma|Total|Lung|Male|Aged|Liver|Drug Therapy|Dental': '2084N0400X',
 'Rehabilitation|Transplantation|Clinic

In [33]:
# creating keyword taxonomy map to find similar keywords using similarity techniques for the kywods that doesnt match exactly
keyword_taxonomy_map

{'Dental': ['2084P0800X',
  '2084N0400X',
  '207R00000X',
  '2084P0804X',
  '207Q00000X',
  '363AM0700X',
  '207RC0000X',
  '207N00000X',
  '390200000X',
  '174400000X',
  '207T00000X',
  '363A00000X',
  '2083X0100X',
  '2084P2900X',
  '363LF0000X',
  '207RH0000X',
  '208D00000X',
  '208000000X',
  '207W00000X',
  '364SP0809X',
  '207RG0300X',
  '2084S0012X',
  '2084F0202X',
  '207RX0202X',
  '207RH0003X',
  '207P00000X',
  '207YX0901X',
  nan,
  '207RE0101X',
  '207RR0500X',
  '204D00000X',
  '207RP1001X',
  '2084N0008X',
  '207RG0100X',
  '163W00000X',
  '207VM0101X',
  '207RA0401X',
  '2084N0402X',
  '208M00000X',
  '2084N0600X',
  '2084P0015X',
  '2080P0216X',
  '2084P0802X',
  '2085R0202X',
  '363LP0200X',
  '2084V0102X',
  '207RA0000X',
  '2080P0207X',
  '2084A0401X',
  '363L00000X',
  '207V00000X',
  '363LA2200X',
  '2084P0805X',
  '207X00000X',
  '208VP0000X',
  '363LP0808X',
  '207RC0200X',
  '207ZN0500X',
  '2080P0206X',
  '207ZP0213X',
  '2084A2900X',
  '2080P0008X',
  '2085

In [34]:
# classification model training
# dropping url and keywords column from test
test=test.drop(["URL","KEYWORDS"],axis=1)

In [35]:
keyword_vector 

{'General': array([ 0.07425748,  0.2148097 , -0.14351544, -0.1586284 ,  0.16520478,
        -0.05241118,  0.21472281,  0.61058503, -0.15232867, -0.07702728,
         0.1878464 , -0.32652465, -0.00194048,  0.12030993,  0.2531358 ,
         0.08998472, -0.00287741, -0.21643497, -0.17373465, -0.5292196 ,
         0.23533991,  0.18187223,  0.13533615, -0.1505036 , -0.29768714,
         0.01720751, -0.3599374 , -0.07317765,  0.05987115,  0.11408166,
         0.18571395, -0.23589739, -0.13072073, -0.10345285, -0.3092688 ,
        -0.05333584, -0.00924568, -0.17556658,  0.22027914, -0.43930104,
         0.45905635, -0.03990025,  0.00793559,  0.16866645,  0.09772234,
        -0.10728482, -0.37835082, -0.08250306,  0.08714733,  0.12988125,
         0.07140333,  0.05834563,  0.04783516, -0.00852136, -0.07600785,
        -0.10944397,  0.49491173,  0.05769522, -0.13067205,  0.1497436 ,
         0.02546669,  0.02949462, -0.05040802, -0.04017427, -0.23269947,
         0.4434914 , -0.00581825,  0.221

In [36]:
# Creating a feature matrix for modelling
keyword_vector_length = len(df['KEYWORD_VECTORS'].iloc[0])
vector_length = keyword_vector_length  # Assuming 'vector_length' is the correct length of the keyword vectors

# Adjust the shape of the feature matrix
feature_matrix = np.zeros((len(df), keyword_vector_length + vector_length))

# Populate the feature matrix
for i, keywords in enumerate(df['PREPROCESSED_KEYWORDS']):
    keyword_vectors = []  # Initialize the keyword vectors list for each instance
    keywords_split = keywords.split("|")
    for keyword in keywords_split:
        if keyword in keyword_vector:
            vector = keyword_vector[keyword]  # Retrieve the keyword vector from the mapping
            keyword_vectors.append(vector)  # Add the 'KEYWORD_VECTORS' value to the feature matrix
    if keyword_vectors:
        feature_matrix[i, :keyword_vector_length] = np.mean(keyword_vectors, axis=0)  # Use mean pooling for keyword vectors
    feature_matrix[i, keyword_vector_length:] = df['KEYWORD_VECTORS'].iloc[i]
feature_matrix[11]

array([ 1.29787484e-02,  2.63470918e-01, -1.49894074e-01, -1.55649036e-01,
        1.91131324e-01, -1.85085878e-01,  2.88845748e-01,  5.73792696e-01,
       -2.43527561e-01, -8.32678527e-02,  1.33740634e-01, -2.87062466e-01,
       -7.95930475e-02,  2.09755916e-02,  2.29526624e-01,  7.94401094e-02,
        2.44743116e-02, -1.71269864e-01, -1.82655752e-01, -5.56795835e-01,
        2.67728955e-01,  4.31826264e-02,  8.06151554e-02, -1.29297659e-01,
       -1.55482620e-01, -1.02833070e-01, -2.53404498e-01, -8.35668743e-02,
        4.59952913e-02,  1.58616379e-01,  2.02922747e-01, -2.91089088e-01,
       -1.86091959e-01, -1.46377593e-01, -2.90437400e-01,  4.87971632e-03,
       -4.55936563e-04, -2.06974059e-01,  2.02424139e-01, -3.47148567e-01,
        3.09323490e-01, -2.97725499e-02,  5.78313693e-02,  2.33529225e-01,
        2.75378764e-01,  1.07984450e-02, -2.28274763e-01,  3.09275114e-03,
        1.35425970e-01,  8.15625489e-02,  5.87633289e-02,  2.76233885e-03,
        8.09575245e-02, -

In [37]:
# feature matrix for test data
keyword_vector_length1 = len(test['KEYWORD_VECTORS'].iloc[0])
vector_length1 = keyword_vector_length1  # Assuming 'vector_length1' is the correct length of the keyword vectors

# Adjust the shape of the feature matrix
feature_matrix1 = np.zeros((len(test), keyword_vector_length1 + vector_length1))

# Populate the feature matrix
for i, keywords in enumerate(test['PREPROCESSED_KEYWORDS']):
    keyword_vectors1 = []  # Initialize the keyword vectors list for each instance
    keywords_split = keywords.split("|")
    for keyword in keywords_split:
        if keyword in keyword_vector1:
            vector = keyword_vector1[keyword]  # Retrieve the keyword vector from the mapping
            keyword_vectors1.append(vector)
    
    if keyword_vectors1:
        feature_matrix1[i, :keyword_vector_length1] = np.mean(keyword_vectors1, axis=0)  # Use mean pooling for keyword vectors
    feature_matrix1[i, keyword_vector_length1:] = test['KEYWORD_VECTORS'].iloc[i]  # Add the 'KEYWORD_VECTORS' value to the feature matrix

feature_matrix1.shape 

(28493, 200)

In [38]:
# added all necessary columns into feature matrix for modelling purposes
one_hot_encoder = OneHotEncoder(sparse=False)
device_type_encoded = one_hot_encoder.fit_transform(df['DEVICETYPE'].values.reshape(-1, 1))

label_encoder = LabelEncoder()
platform_type_encoded = label_encoder.fit_transform(df['PLATFORMTYPE'])

url_terms_counts = df['url_terms'].value_counts().to_dict()
df['url_terms_encoded'] = df['url_terms'].map(url_terms_counts)

additional_features = np.concatenate([device_type_encoded, platform_type_encoded.reshape(-1, 1), df['url_terms_encoded'].values.reshape(-1, 1)], axis=1)

feature_matrix = np.concatenate([feature_matrix, additional_features], axis=1)


feature_matrix.shape

C:\Users\P.Vijay Srinivasan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(113936, 206)

In [39]:
# # added all necessary columns into feature matrix for test data
one_hot_encoder1 = OneHotEncoder(sparse=False)
device_type_encoded1 = one_hot_encoder1.fit_transform(test['DEVICETYPE'].values.reshape(-1, 1))

# Label Encoding for 'platform_type'
label_encoder1 = LabelEncoder()
platform_type_encoded1 = label_encoder1.fit_transform(test['PLATFORMTYPE'])

# Count Encoding for 'url_terms'
url_terms_counts1 = test['url_terms'].value_counts().to_dict()
test['url_terms_encoded'] = test['url_terms'].map(url_terms_counts1)

# Combine encoded features and create the feature matrix
additional_features1= np.concatenate([device_type_encoded1, platform_type_encoded1.reshape(-1, 1), test['url_terms_encoded'].values.reshape(-1, 1)], axis=1)

# Update the feature matrix by concatenating with additional features
feature_matrix1 = np.concatenate([feature_matrix1, additional_features1], axis=1)
# feature_matrix1 =feature_matrix1.concatenate([device_type_encoded1, platform_type_encoded1.reshape(-1, 1), test['url_terms_encoded'].values.reshape(-1, 1)], axis=1)

# Split the test into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(feature_matrix, test['is_hcp'], test_size=0.2, random_state=42)

# Train your classification model using the prepared test
# ...

feature_matrix1

C:\Users\P.Vijay Srinivasan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[-8.82534832e-02,  2.55432874e-01,  4.28568125e-02, ...,
         0.00000000e+00,  3.00000000e+00,  1.35200000e+03],
       [-8.53537321e-02,  2.78070927e-01,  3.50422226e-02, ...,
         0.00000000e+00,  3.00000000e+00,  1.30000000e+01],
       [-8.82534832e-02,  2.55432874e-01,  4.28568125e-02, ...,
         0.00000000e+00,  3.00000000e+00,  1.06000000e+02],
       ...,
       [-8.32253098e-02,  2.76695520e-01,  2.65371166e-02, ...,
         0.00000000e+00,  3.00000000e+00,  5.00000000e+00],
       [-8.54355842e-02,  2.34284103e-01,  2.78704893e-02, ...,
         0.00000000e+00,  3.00000000e+00,  4.00000000e+00],
       [-8.10805708e-02,  2.43332952e-01,  2.62160823e-02, ...,
         0.00000000e+00,  3.00000000e+00,  7.00000000e+00]])

In [40]:
#Preparing data for modelling
X_train=feature_matrix
y=df["IS_HCP"]
# splitting data into train and validation
x_train,x_val,y_train, y_val =train_test_split(X_train, y, test_size=0.25, random_state=42)

x_test=feature_matrix1

In [41]:
# using RF classifier
rf=RandomForestClassifier(n_estimators=390,max_depth=20,class_weight='balanced',min_samples_split=11,min_samples_leaf=10,max_features='sqrt',max_leaf_nodes=50,n_jobs=-1,random_state=42)
rf.fit(x_train,y_train)

RandomForestClassifier(class_weight='balanced', max_depth=20, max_leaf_nodes=50,
                       min_samples_leaf=10, min_samples_split=11,
                       n_estimators=390, n_jobs=-1, random_state=42)

In [42]:
# predicting validation values
y_pred_val = rf.predict(x_val)
y_pred_val

array([1., 0., 0., ..., 0., 0., 0.])

In [43]:
# evaluating validation values

print(classification_report(y_val, y_pred_val))

              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96     19928
         1.0       0.87      0.95      0.91      8556

    accuracy                           0.94     28484
   macro avg       0.92      0.95      0.93     28484
weighted avg       0.95      0.94      0.94     28484



In [44]:
# # predicting train and test values

train_pred=rf.predict(x_train)
y_pred=rf.predict(x_test)
train_pred

array([1., 0., 0., ..., 0., 1., 0.])

In [45]:
val_df=pd.DataFrame(y_pred_val)
val_df.rename(columns={0:"is_hcp or not"},inplace=True)
val_df[val_df["is_hcp or not"]==1]

,is_hcp or not
0,1.0
5,1.0
8,1.0
12,1.0
14,1.0
...,...
28457,1.0
28464,1.0
28467,1.0
28470,1.0


In [46]:
# evaluating train values

print(classification_report(y_train,train_pred))

              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96     59828
         1.0       0.86      0.95      0.91     25624

    accuracy                           0.94     85452
   macro avg       0.92      0.94      0.93     85452
weighted avg       0.94      0.94      0.94     85452



In [47]:
confusion_matrix(y_pred_val,y_val)

array([[18671,   394],
       [ 1257,  8162]], dtype=int64)

In [48]:
# CREATING FINAL TEST DATA FOR USER ID'S WITH PREDICTED VALUES
# index=test.index
test=test.reset_index()
final_df=pd.DataFrame(y_pred,index=test["ID"])
final_df.rename(columns={0:"is_hcp or not"},inplace=True)
final_df[final_df["is_hcp or not"]==1]
# Assign the predicted values to the DataFrame column
# predicted_df['Predicted_Value'] = predicted_test
# final_df
test.set_index('ID', inplace=True)
# final_df.set_index('ID', inplace=True)
final_df['KEYWORDS'] = test['PREPROCESSED_KEYWORDS']
final_df['KEYWORD_VECTORS'] = test['KEYWORD_VECTORS']

In [49]:
final_df[final_df["is_hcp or not"]==1]

,is_hcp or not,KEYWORDS,KEYWORD_VECTORS
ID,,,
115502,1.0,Bone Marrow|Radiography|Chronic|Oncology|Psych...,"[-0.03819745, 0.37528032, 0.019585242, 0.24406..."
115503,1.0,General|Endocrine|False|Medicine|Surgery|Urolo...,"[-0.06629049, 0.2323176, 0.13064353, 0.2678785..."
115505,1.0,Critical Care|Emergency Medicine|General Pract...,"[-0.041238148, 0.3103347, 0.053170558, 0.19120..."
115507,1.0,Transplantation|Technology|Chronic|Neoplasms|U...,"[-0.13984418, 0.27199566, -0.04747326, 0.12852..."
115508,1.0,Family Practice|Drainage|Clinical|Dermatology|...,"[-0.06912753, 0.2191029, -0.055128083, 0.25517..."
...,...,...,...
143989,1.0,General|Neurology|Ophthalmology|Medicine|Clini...,"[-0.0994343, 0.23626044, 0.063134715, 0.276399..."
143990,1.0,Oncology|Medicine|Neurology|Ophthalmology|Pedi...,"[-0.09712157, 0.2580904, 0.030989867, 0.272204..."
143991,1.0,Diagnosis|Pathology|Neurological Surgery|Inten...,"[-0.053196255, 0.32650667, 0.07632161, 0.14086..."


In [50]:
# FILLING TAXONOMIES WHERE KEYWORD MATCHES WITH THE TAXONOMY_MAP
final_df['taxonomy'] =final_df.apply(lambda row: taxonomy_map[row['KEYWORDS']] if row['is_hcp or not'] == 1 and row['KEYWORDS'] in taxonomy_map else 'no taxonomy', axis=1)
final_df

,is_hcp or not,KEYWORDS,KEYWORD_VECTORS,taxonomy
ID,,,,
115501,0.0,General|Endocrine|False|Medicine|Surgery|Urolo...,"[-0.06629049, 0.2323176, 0.13064353, 0.2678785...",no taxonomy
115502,1.0,Bone Marrow|Radiography|Chronic|Oncology|Psych...,"[-0.03819745, 0.37528032, 0.019585242, 0.24406...",no taxonomy
115503,1.0,General|Endocrine|False|Medicine|Surgery|Urolo...,"[-0.06629049, 0.2323176, 0.13064353, 0.2678785...",NaN
115504,0.0,Dental|Total|Clinical|Pharmacology|Physicians|...,"[-0.008534272, 0.36169952, 0.016364213, 0.3606...",no taxonomy
115505,1.0,Critical Care|Emergency Medicine|General Pract...,"[-0.041238148, 0.3103347, 0.053170558, 0.19120...",no taxonomy
...,...,...,...,...
143989,1.0,General|Neurology|Ophthalmology|Medicine|Clini...,"[-0.0994343, 0.23626044, 0.063134715, 0.276399...",207Q00000X
143990,1.0,Oncology|Medicine|Neurology|Ophthalmology|Pedi...,"[-0.09712157, 0.2580904, 0.030989867, 0.272204...",2084P0800X
143991,1.0,Diagnosis|Pathology|Neurological Surgery|Inten...,"[-0.053196255, 0.32650667, 0.07632161, 0.14086...",no taxonomy


In [51]:
final_df["taxonomy"].astype('str')

ID
115501    no taxonomy
115502    no taxonomy
115503            nan
115504    no taxonomy
115505    no taxonomy
             ...     
143989     207Q00000X
143990     2084P0800X
143991    no taxonomy
143992     207R00000X
143993    no taxonomy
Name: taxonomy, Length: 28493, dtype: object

In [52]:
keyword_vector1

{'General': array([-0.0994343 ,  0.23626044,  0.06313471,  0.27639925, -0.18696675,
        -0.3181883 ,  0.15480773,  0.73681253, -0.45350197, -0.18834822,
         0.0285441 , -0.18170506, -0.14215714,  0.15110625,  0.07180658,
         0.17822994,  0.39513856, -0.06081693, -0.03061447, -0.5768645 ,
        -0.05769561,  0.03800605, -0.03569662, -0.04284023, -0.33275023,
         0.21526022, -0.16476554,  0.05398061, -0.11049511,  0.15334065,
         0.3505815 , -0.03092033,  0.04594718, -0.14517862, -0.18507771,
         0.04810113,  0.17230065,  0.03759353,  0.01831684, -0.24803795,
         0.25081295, -0.1243788 , -0.05821806,  0.14398283,  0.11787495,
        -0.20521384, -0.37656847, -0.06776101,  0.16174199, -0.2437647 ,
         0.01016431, -0.04346409, -0.0588652 , -0.04852492,  0.14236718,
        -0.19355497,  0.2699836 , -0.22769907, -0.00936311,  0.17008941,
        -0.2572377 ,  0.2276286 ,  0.2340551 , -0.09703996, -0.16540764,
         0.5159995 ,  0.02229549,  0.244

In [53]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(vector1, vector2):
    return cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))[0][0]

num_neighbors = 5  # Number of neighbors to consider

# Iterate over each row in the dataframe
for index, row in final_df.iterrows():
    if row['is_hcp or not'] == 1 and (pd.isnull(row['taxonomy']) or row['taxonomy'] == 'no taxonomy'):
        keyword_vector = row['KEYWORD_VECTORS']
        keyword_scores = {}
        
        # Calculate similarity scores with each keyword in the keyword_taxonomy_map
        for keyword, taxonomies in keyword_taxonomy_map.items():
            similarities = []
            if keyword in keyword_vector1:
                for taxonomy in taxonomies:
                    similarity = calculate_similarity(keyword_vector, keyword_vector1[keyword])
                    similarities.append(similarity)
            keyword_scores[keyword] = max(similarities) if similarities else 0.0
        
        # Sort keywords by similarity score in descending order
        sorted_keywords = sorted(keyword_scores, key=keyword_scores.get, reverse=True)
        
        # Find the top similar keywords with known taxonomies
        top_similar_keywords = [k for k in sorted_keywords if k in keyword_taxonomy_map][:num_neighbors]
        
        # Get the taxonomies of the top similar keywords
        top_taxonomies = [taxonomy for keyword in top_similar_keywords for taxonomy in keyword_taxonomy_map[keyword]]
        
        # Find the most common taxonomy among the top neighbors
        most_common_taxonomy = max(set(top_taxonomies), key=top_taxonomies.count)
        
        # Update the taxonomy for the current row
        final_df.at[index, 'taxonomy'] = most_common_taxonomy
        print(f"Updated Taxonomy for index {index}: {most_common_taxonomy}")


Updated Taxonomy for index 115502: 2084P0800X
Updated Taxonomy for index 115503: 2084N0008X
Updated Taxonomy for index 115505: 2084P0800X
Updated Taxonomy for index 115507: 2084P0800X
Updated Taxonomy for index 115508: 2084P0800X
Updated Taxonomy for index 115511: 2084P0800X
Updated Taxonomy for index 115512: 2084P0800X
Updated Taxonomy for index 115513: 2084P0800X
Updated Taxonomy for index 115514: 207Y00000X


KeyboardInterrupt: 

In [ ]:
final_df[final_df["taxonomy"]=='no taxonomy'].head(50)

In [54]:
final_df=final_df.drop('KEYWORD_VECTORS',axis=1)

In [55]:
final_df[final_df["taxonomy"]!="no taxonomy"].head(50)

,is_hcp or not,KEYWORDS,taxonomy
ID,,,
115502,1.0,Bone Marrow|Radiography|Chronic|Oncology|Psych...,2084P0800X
115503,1.0,General|Endocrine|False|Medicine|Surgery|Urolo...,2084N0008X
115505,1.0,Critical Care|Emergency Medicine|General Pract...,2084P0800X
115507,1.0,Transplantation|Technology|Chronic|Neoplasms|U...,2084P0800X
115508,1.0,Family Practice|Drainage|Clinical|Dermatology|...,2084P0800X
115509,1.0,Intensive Care|Shock|Ischemic Heart Disease|He...,2084P0804X
115511,1.0,Medicine|Transplantation|Clinical|Trauma|Total...,2084P0800X
115512,1.0,Family Practice|Drainage|Clinical|Dermatology|...,2084P0800X
115513,1.0,Pharmacology|Resuscitation|General Practice|Ph...,2084P0800X


In [ ]:
final_df.drop(["KEYWORDS","taxonomy"],axis=1)

In [57]:
final_df=final_df.to_csv("Final_output_file2.csv",index=True)

In [ ]:
# final_df[(final_df["taxonomy"]!="no taxonomy")]
# Bone Marrow|Radiography|Chronic|Oncology|Psychiatry|Intestine|Small Intestines|Physicians|Gastroenterology|Salivary Glands|Autoimmune|Hemolytic|False|Hereditary|Total|Cardiology|Surgery|Clinical|Dendritic Cells|Lung|Biopsy|General|Large|Small|Ophthalmology|Medicine|Lymphoid Tissue|Rheumatology|BLymphocytes|Stomach|Transplantation|Gastrointestinal Tract|Intestines|Diagnosis|Endocrine|Anemia|Refractory|Dermatology|Neurology|Urology
taxonomy_map

In [ ]:
# Performing k-fold cross-validation with multiple scoring metrics
cv_results1 = cross_val_score(rf, x_train, y_train, cv=5, scoring='accuracy')
cv_results2 = cross_val_score(rf, x_train, y_train, cv=5, scoring='precision')
# cv_results3= cross_val_score(rf, x_train, y_train, cv=5, scoring='recall')
# cv_results4 = cross_val_score(rf, x_train, y_train, cv=5, scoring='f1')

# mean_accuracy = cv_results1.mean()
# mean_precision = cv_results2.mean()
# mean_recall = cv_results3.mean()
# mean_f1_score = cv_results4.mean()

# std_accuracy = cv_results1.std()
# std_precision = cv_results2.std()
# std_recall = cv_results3.std()
# std_f1_score = cv_results4.std()

# print("Accuracy: {:.2f} (+/- {:.2f})".format(mean_accuracy, std_accuracy))
# print("Precision: {:.2f} (+/- {:.2f})".format(mean_precision, std_precision))
# print("Recall: {:.2f} (+/- {:.2f})".format(mean_recall, std_recall))
# print("F1 Score: {:.2f} (+/- {:.2f})".format(mean_f1_score, std_f1_score))
cv_results2

In [ ]:
def fill_taxonomy(hcp_instances, taxonomy_map):
    taxonomy_values = []

    for instance in hcp_instances:
        similar_keywords = get_similar_keywords(instance, taxonomy_map)

        # Calculate cumulative similarity scores for each taxonomy value
        taxonomy_scores = {}
        for keyword, taxonomy in similar_keywords.items():
            similarity_score = model.wv.similarity(instance, keyword)
            if taxonomy in taxonomy_scores:
                taxonomy_scores[taxonomy] += similarity_score
            else:
                taxonomy_scores[taxonomy] = similarity_score

        # Find the taxonomy value with the highest cumulative similarity score
        if taxonomy_scores:
            best_taxonomy = max(taxonomy_scores, key=taxonomy_scores.get)
            taxonomy_values.append((instance, best_taxonomy, taxonomy_scores[best_taxonomy]))

    return taxonomy_values

def get_similar_keywords(keywords, taxonomy_map):
    similar_keywords = {}
    for keyword in keywords.split('|'):
        if keyword in taxonomy_map:
            taxonomy = taxonomy_map[keyword]
            similar_keywords[keyword] = taxonomy

    return similar_keywords

# Assuming you have a list of HCP instances
hcp_instances = hcp_without_taxonomy[hcp_without_taxonomy['IS_HCP'] == 1]['PREPROCESSED_KEYWORDS'].tolist()

# Assuming you have a dataset with keywords and their taxonomy values
# Replace `keyword_taxonomy_map` with your actual keyword-to-taxonomy dictionary

# Fill taxonomy values for HCP instances
hcp_taxonomy_values = fill_taxonomy(hcp_instances, keyword_taxonomy_map)
print(hcp_taxonomy_values)
hcp_instances

In [ ]:
keyword_taxonomy_map

In [ ]:
# for i in taxonomy_map.values():
#     print(type(i))
# Check data type compatibility
if df['KEYWORDS'].dtype != str:
    df['KEYWORDS'] = df['KEYWORDS'].astype(str)

# Verify presence of exact matches
exact_matches = set(df['KEYWORDS']).intersection(taxonomy_map.keys())

print("Exact Matches Found:")
print(len(exact_matches))


In [ ]:
# Filter rows where HCP is identified but taxonomy is not populated
hcp_without_taxonomy = df[(df['TAXONOMY'].isnull())& (df['IS_HCP'] == True)]

# Filter rows where both HCP and taxonomy are identified
hcp_with_taxonomy = df[(df['IS_HCP'] == True) & (df['TAXONOMY'].notnull())]

# Filter rows where there is no information about HCP
unknown_hcp = df[df['IS_HCP'] != True]
hcp_with_taxonomy